<a href="https://colab.research.google.com/github/jiin124/Dacon/blob/main/%ED%95%9C%EA%B5%AD%EC%96%B4%20%EB%AC%B8%EC%9E%A5%20%EA%B4%80%EA%B3%84%20%EB%B6%84%EB%A5%98%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/%ED%95%9C%EA%B5%AD%EC%96%B4_%EB%AC%B8%EC%9E%A5_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from glob import glob
import os
import numpy as np
from tqdm import tqdm, tqdm_notebook


import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW

In [ ]:
pip install gluonnlp

In [ ]:
pip install mxnet

In [ ]:
pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [40]:
train=pd.read_csv('/content/train_data.csv')
test=pd.read_csv('/content/test_data.csv')
sample=pd.read_csv('/content/sample_submission.csv')

In [41]:
train.head()

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral


In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24998 entries, 0 to 24997
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       24998 non-null  int64 
 1   premise     24998 non-null  object
 2   hypothesis  24998 non-null  object
 3   label       24998 non-null  object
dtypes: int64(1), object(3)
memory usage: 781.3+ KB


In [43]:
train['label'].value_counts()

entailment       8561
contradiction    8489
neutral          7948
Name: label, dtype: int64

In [44]:
test.head()

,index,premise,hypothesis,label
0,0,다만 조금 좁아서 케리어를 펼치기 불편합니다.,케리어를 펼치기에 공간이 충분했습니다.,answer
1,1,그리고 위치가 시먼역보다는 샤오난먼역에 가까워요,시먼역보다는 샤오난먼역에 먼저 도착할 수 있어요,answer
2,2,구구절절 설명하고 이해시키려는 노력이 큰 의미없이 다가온다.,무엇인가 말을 많이 하기는 했지만 큰 의미가 있지는 않았다.,answer
3,3,몇 번을 다시봐도 볼 때마다 가슴이 저민다.,다시 봤을때는 무덤덤했다.,answer
4,4,"8월 중에 입주신청을 하면 청년은 9월, 신혼부부는 10월부터 입주가 가능하다.",8월 중에 입주신청을 하면 신혼부부는 9월 부터 입주가 가능하다.,answer


In [39]:
sample.head()

,index,label
0,0,answer
1,1,answer
2,2,answer
3,3,answer
4,4,answer


In [22]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device('cpu')


In [23]:
max_len = 70
batch_size = 64    # Batch size 감소
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

## 토큰화, 패딩, 어텐션 마스크, 배치 및 데이터로더 설정

In [45]:
bertmodel,vocab=get_pytorch_kobert_model(cachedir='.cache')

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [63]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [64]:
label_dict = {"entailment" : 0, "contradiction" : 1, "neutral" : 2}

In [62]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sen_idx, label_idx, bert_tokenizer, max_len,
                pad, pair, mode = "train"):
        self.mode = mode
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length = max_len, pad = pad, pair = pair)
        if self.mode == "train":
            self.sentence = [transform(i[sen_idx]) for i in dataset]   # pair=True를 위해 수정. [] 제거
            self.labels = [np.int32(i[label_idx]) for i in dataset]
            
        else:
            self.sentence =  [transform(i[sen_idx]) for i in dataset]   # pair=True를 위해 수정. [] 제거
        
    def __getitem__(self, i):
        if self.mode == 'train':
            return (self.sentence[i] +  (self.labels[i], ))
        else:
            return self.sentence[i]
    
    def __len__(self):
        return (len(self.sentence))

BERTdataset 클래스는 tsv dataset과 dataset의 sent_idx,dataset의 label_idx,버트 토크나이저,max_len,pad와 pair에 대한 bool값을 인자로 받아 토큰화, 패딩을 하고 어텐션 마스크를 만들어 inputs, labels, masks를 TensorDataset으로 변환해 뱉어준다. 

# Tokenizing

- Baseline 코드를 돌려보던 중 Tokenizing에서 이상한 점 발견
- [CLS]와 [SEP]가 토큰으로 인식되지 않고 분해되는 것을 발견
- Fair를 활용

In [61]:
# [CLS], [SEP] 토큰 분해
text1 = train['premise'][0]
text2 = train['hypothesis'][0]
concat_text = "[CLS]" + text1 + "[SEP] " + text2 + "[SEP]"
tok(concat_text)

['▁[',
 '▁C',
 'LS',
 '▁',
 ']',
 '▁씨',
 '름',
 '은',
 '▁상',
 '고',
 '시대',
 '로부터',
 '▁전해',
 '져',
 '▁내려',
 '오는',
 '▁남자',
 '들의',
 '▁대표적인',
 '▁놀',
 '이',
 '로서',
 '▁',
 ',',
 '▁소',
 '년',
 '이나',
 '▁장',
 '정',
 '들이',
 '▁넓',
 '고',
 '▁평',
 '평',
 '한',
 '▁백',
 '사',
 '장',
 '이나',
 '▁마',
 '당',
 '에서',
 '▁모여',
 '▁서로',
 '▁힘',
 '과',
 '▁슬',
 '기를',
 '▁',
 '겨',
 '루',
 '는',
 '▁것이다',
 '▁',
 '.',
 '▁[',
 '▁S',
 'E',
 'P',
 '▁',
 ']',
 '▁씨',
 '름',
 '의',
 '▁여자',
 '들의',
 '▁놀',
 '이',
 '이다',
 '▁',
 '.',
 '▁[',
 '▁S',
 'E',
 'P',
 '▁',
 ']']

In [60]:

transform = nlp.data.BERTSentenceTransform(tok, max_seq_length = max_len, pad = True, pair = False)

# [CLS]와 [SEP]는 자동으로 붙여주는거 같음
transform(concat_text)

(array([  2, 702,   3,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1], dtype=int32),
 array(3, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0], dtype=int32))

In [59]:
# Fair를 활용한 tokenizing
text1 = train['premise'][0]
text2 = train['hypothesis'][0]

# pair True 설정
transform = nlp.data.BERTSentenceTransform(tok, max_seq_length = max_len, pad = True, pair = True)
transform([text1, text2])

(array([   2, 3088, 6117, 7086, 2658, 5439, 6708, 6080, 4059, 7245, 1442,
        6965, 1423, 5939, 1678, 1504, 7096, 6081,  517,   46, 2822, 5712,
        7098, 3954, 7227, 5940, 1459, 5439, 4841, 7724, 7828, 2298, 6493,
        7178, 7098, 1907, 5804, 6903, 2064, 2720, 5211, 5468, 2948, 5573,
         517, 5411, 6095, 5760,  913,  517,   54,    3, 3088, 6117, 7095,
        3318, 5939, 1504, 7096, 7100,  517,   54,    3,    1,    1,    1,
           1,    1,    1,    1], dtype=int32),
 array(63, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 0], dtype=int32))

In [69]:
train["premise_"] = "[CLS]" + train["premise"] + "[SEP]"
train["hypothesis_"] = train["hypothesis"] + "[SEP]"

test["premise_"] = "[CLS]" + test["premise"] + "[SEP]"
test["hypothesis_"] = test["hypothesis"] + "[SEP]"

train["text_sum"] = train.premise_ + " " + train.hypothesis_
test["text_sum"] = test.premise_ + " " + test.hypothesis_

train_content = []
test_content = []

for i, text in enumerate(train.text_sum):
    train_content.append(list([text, str(label_dict[train.label[i]])]))
    
for i, text in enumerate(test.text_sum):
    test_content.append([text])
    
dataset_train = train_content[:20000]
dataset_valid = train_content[20000:]
dataset_test = test_content

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False, mode = "train")
data_valid = BERTDataset(dataset_valid, 0, 1, tok, max_len, True, False, mode = "train")
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False, mode = "test")

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
valid_dataloader = torch.utils.data.DataLoader(data_valid, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [67]:
train_dataloader

# 모델만들기

In [70]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size = 768, num_classes=3, dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [71]:
model = BERTClassifier(bertmodel, dr_rate = 0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [72]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    valid_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        train_acc += calc_accuracy(out, label)

    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(valid_dataloader), total=len(valid_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        valid_acc += calc_accuracy(out, label)
    print("epoch {} valid acc {}".format(e+1, valid_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 85%|████████▍ | 266/313 [2:56:22<31:19, 39.98s/it]

In [ ]:
result = []
model.eval()
with torch.no_grad():
    for batch_id, (token_ids, valid_length, segment_ids) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        result.append(model(token_ids, valid_length, segment_ids))

In [ ]:

result_ = []
for i in result:
    for j in i:
        result_.append(int(torch.argmax(j)))
        
out = [list(label_dict.keys())[_] for _ in result_]

sample["label"] = out

submission.to_csv("sample_submission.csv", index = False)